In [2]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import seaborn as sns
from utils.pddataframes import (
    print_ocurrencies, plot_barchar, plot_multiple_barchar, get_high_season, get_period_day,
    plot_grouped_barchar
)
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, roc_curve, accuracy_score, confusion_matrix,  precision_recall_fscore_support

synthetic_features_csv = pd.read_csv("synthetic_features.csv")

columns_names = list(synthetic_features_csv)
synthetic_features_csv.head()

C:\Users\Felipe\AppData\Local\Temp/ipykernel_31368/3720199085.py:14: DtypeWarning: Columns (1,5) have mixed types. Specify dtype option on import or set low_memory=False.
  synthetic_features_csv = pd.read_csv("synthetic_features.csv")


,Fecha-I,Vlo-I,Des-I,Emp-I,Fecha-O,Vlo-O,Des-O,Emp-O,DIA,MES,DIANOM,TIPOVUELO,OPERA,SIGLADES,high_season,min_diff,delay_15,period_day
0,2017-01-01 23:30:00,226,KMIA,AAL,2017-01-01 23:33:00,226,KMIA,AAL,1,1,Domingo,I,American Airlines,Miami,1,3.0,0,night
1,2017-01-02 23:30:00,226,KMIA,AAL,2017-01-02 23:39:00,226,KMIA,AAL,2,1,Lunes,I,American Airlines,Miami,1,9.0,0,night
2,2017-01-03 23:30:00,226,KMIA,AAL,2017-01-03 23:39:00,226,KMIA,AAL,3,1,Martes,I,American Airlines,Miami,1,9.0,0,night
3,2017-01-04 23:30:00,226,KMIA,AAL,2017-01-04 23:33:00,226,KMIA,AAL,4,1,Miercoles,I,American Airlines,Miami,1,3.0,0,night
4,2017-01-05 23:30:00,226,KMIA,AAL,2017-01-05 23:28:00,226,KMIA,AAL,5,1,Jueves,I,American Airlines,Miami,1,-2.0,0,night


In [100]:
#Remove useles columns
from sklearn.preprocessing import OneHotEncoder
training_dataset = synthetic_features_csv.drop(['Fecha-O', 'Vlo-O', 'Des-O', 'Emp-O', 'min_diff', 'OPERA', 'Vlo-I'], axis = 1)
training_dataset = training_dataset.assign(fechats = lambda df: pd.to_datetime(
    df['Fecha-I'], format='%Y-%m-%d %H:%M:%S').astype(np.int64) // 10**9)
training_dataset = training_dataset.drop(['Fecha-I'], axis = 1)
training_dataset.head()
training_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68205 entries, 0 to 68204
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Des-I        68205 non-null  object
 1   Emp-I        68205 non-null  object
 2   DIA          68205 non-null  int64 
 3   MES          68205 non-null  int64 
 4   DIANOM       68205 non-null  object
 5   TIPOVUELO    68205 non-null  object
 6   SIGLADES     68205 non-null  object
 7   high_season  68205 non-null  int64 
 8   delay_15     68205 non-null  int64 
 9   period_day   68205 non-null  object
 10  fechats      68205 non-null  int64 
dtypes: int64(5), object(6)
memory usage: 5.7+ MB


In [77]:
training_dataset = training_dataset[['Des-I', 'Emp-I', 'DIANOM', 'TIPOVUELO', 'SIGLADES', 'period_day', 'DIA', 'MES', 'high_season', 'fechats', 'delay_15']]

training_dataset_columns = list(training_dataset)
print(training_dataset_columns)
training_dataset.head()

['Des-I', 'Emp-I', 'DIANOM', 'TIPOVUELO', 'SIGLADES', 'period_day', 'DIA', 'MES', 'high_season', 'fechats', 'delay_15']


,Des-I,Emp-I,DIANOM,TIPOVUELO,SIGLADES,period_day,DIA,MES,high_season,fechats,delay_15
0,KMIA,AAL,Domingo,I,Miami,night,1,1,1,1483313400,0
1,KMIA,AAL,Lunes,I,Miami,night,2,1,1,1483399800,0
2,KMIA,AAL,Martes,I,Miami,night,3,1,1,1483486200,0
3,KMIA,AAL,Miercoles,I,Miami,night,4,1,1,1483572600,0
4,KMIA,AAL,Jueves,I,Miami,night,5,1,1,1483659000,0


In [86]:
categorical_columns_numpy = training_dataset.values[:,0:6]
ohe = OneHotEncoder(sparse_output=False)
ohe.fit(categorical_columns_numpy)
categorical_columns_numpy_enc = ohe.transform(categorical_columns_numpy)

encoded_training_dataset = np.concatenate((categorical_columns_numpy_enc, training_dataset.values[:,6:]), axis=1)
print(encoded_training_dataset.shape)
print(encoded_training_dataset[:,168:])

(68205, 173)
[[1 1 1 1483313400 0]
 [2 1 1 1483399800 0]
 [3 1 1 1483486200 0]
 ...
 [27 12 1 1514386500 1]
 [29 12 1 1514559300 0]
 [31 12 1 1514732100 0]]


In [105]:



train, test = train_test_split(encoded_training_dataset, test_size=.3, random_state=1999)
train_X = train[:,:-1]
train_y = train[:,-1].astype('int')
test_X = test[:,:-1]
test_y = test[:,-1].astype('int')


print(train_X.shape, train_y.shape)
print(test_X.shape, test_y.shape)
print(train_y)

(47743, 172) (47743,)
(20462, 172) (20462,)
[0 0 0 ... 0 0 0]


In [106]:
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=0)
clf.fit(train_X, train_y)


DecisionTreeClassifier(random_state=0)

In [107]:
predictions = clf.predict(test_X)


In [108]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

0.7486071742742645

In [116]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(train_X, train_y)

RandomForestClassifier(max_depth=2, random_state=0)

In [117]:
predictions = clf.predict(test_X)
accuracy_score(test_y, predictions)

0.8123350601114261

In [ ]:
# Plot the feature importances of the forest
plt.figure(figsize=(18,9))
plt.title("Feature importances")
n=20
_ = plt.bar(range(n), importances[indices][:n], color="r", yerr=std[indices][:n])
plt.xticks(range(n), indices)
plt.xlim([-1, n])
plt.show()

In [118]:
from sklearn import svm
clf = svm.SVC()
clf.fit(train_X, train_y)

SVC()

In [119]:
predictions = clf.predict(test_X)
accuracy_score(test_y, predictions)

0.8123350601114261

In [ ]:

from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
iris = datasets.load_iris()
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(iris.data, iris.target)
GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')})
sorted(clf.cv_results_.keys())
['mean_fit_time', 'mean_score_time', 'mean_test_score',...
 'param_C', 'param_kernel', 'params',...
 'rank_test_score', 'split0_test_score',...
 'split2_test_score', ...
 'std_fit_time', 'std_score_time', 'std_test_score']